In [60]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os, string
import csv
import pymongo
import time
import json 
import numpy as np

# Extract (Scrape) Data

#### *Does not need to be loaded to pymongo* - can be scraped on the spot for the Project 2

In [2]:
# currentDirectory = os.path.dirname(__file__)
# executable_path = {"executable_path": f"{currentDirectory}\chromedriver.exe"}
executable_path = {'executable_path': 'chromedriver.exe'} #notebook version
browser = Browser('chrome', **executable_path, headless=True)

### Scrape Schedules

In [59]:
nba_sched_url = "https://www.espn.com/nba/schedule/_/date/"
browser.visit(nba_sched_url)
browser.html
html = browser.html
soup = BeautifulSoup(html, 'lxml')
sched_tables = soup.find('table', class_="schedule has-team-logos align-left")
schedules_sites = pd.read_html(sched_tables.prettify())
schedules_sites

[            matchup          Unnamed: 1 time  (PT) nat tv  \
 0     Portland  POR        Indiana  IND       LIVE    TNT   
 1   Sacramento  SAC  Oklahoma City  OKC       LIVE    NaN   
 2  Los Angeles  LAL    Golden State  GS    7:30 PM    TNT   
 
                  tickets  Unnamed: 5  
 0     Buy on Vivid Seats         NaN  
 1     Buy on Vivid Seats         NaN  
 2  Tickets as low as $36         NaN  ]

# Load - Transform

### Load dump to mongo / Clean on the go

In [133]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.nba
# db.nba_players_stg.drop()
# db.nba_news_live.drop()

In [ ]:
base_reference_url = "https://www.landofbasketball.com"
#Traverse to all players
for letter in string.ascii_lowercase[:26]:
    players_url = f"https://www.landofbasketball.com/nba_players_index/letter_{letter}.htm"
    browser.visit(players_url)
    browser.html
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    all_player_table = soup.find_all('div', class_="indice-items")
    for row in all_player_table:
        player_d = {}
        print(row.find('a')['href'], end='')
        player_url = row.find('a')['href'].replace('..', base_reference_url)
        tables = pd.read_html(player_url)
        try_df = tables[1].set_index(0)
        try_df.rename(columns={0: "Key", 1: "Value"}, inplace=True)
        try_df = try_df.to_dict('index')
        player_d['Name'] = tables[3][0][0].split(' Profile',1)[0]
        player_d['Data'] = try_df
        db.nba_players_stg.insert_one(player_d)

### Validate data availability

In [116]:

# result_dict['facts'] = list(db.mars_fact.find({}, {"_id": 0}))
for a in db.nba_players_stg.find({}, {"_id": 0}).limit(2):
    print(a)

{'Name': 'Alaa Abdelnaby', 'Data': {'Position:': {'Value': 'Forward / Center'}, 'Jersey Number:': {'Value': '31 (Trail Blazers), 5 (Bucks), 4 (Celtics), 30 (Kings, 76ers)'}, 'Height:': {'Value': '6-10 / 2.08 mts'}, 'Weight:': {'Value': '240 lbs / 108.9 kg'}, 'Born:': {'Value': 'June 24, 1968 in Cairo, Egypt'}, 'High-School:': {'Value': 'Bloomfield in Bloomfield, New Jersey'}, 'College:': {'Value': 'Duke University'}, 'Drafted:': {'Value': 'by the Portland Trail Blazers with No. 25 overall pick in round 1 of the 1990 NBA Draft'}, 'NBA Seasons:': {'Value': '5 (3 Playoffs)'}, 'NBA Titles:': {'Value': 'none'}, 'All-Star:': {'Value': 'none'}}}
{'Name': 'Zaid Abdul-Aziz', 'Data': {'Position:': {'Value': 'Center / Forward'}, 'Jersey Number:': {'Value': '21 (Royals), 16 (Bucks), 35 (Bucks, Supersonics), 6 (Rockets), 35 (Rockets, Braves), 54 (Celtics), 27 (Rockets)'}, 'Height:': {'Value': '6-9 / 2.06 mts'}, 'Weight:': {'Value': '235 lbs / 106.6 kg'}, 'Born:': {'Value': 'April 07, 1946 in Brookl

## Scrape News

In [134]:
nba_news_url = "https://apnews.com/NBA"
browser.visit(nba_news_url)
browser.html
# time.sleep(10)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

news_tables = soup.find_all('div', attrs={"data-key":"feed-card-wire-story-with-image"})
# schedules_sites = pd.read_html(sched_tables.prettify())
# schedules_sites


In [ ]:
for row in news_tables:
    news_content = {}
    news_content['headline'] = row.h1.text
    news_content['content'] = row.p.text
    print(f"Loading {row.h1.text}....")
    db.nba_news_live.insert_one(news_content)

### ETL For elections

In [ ]:
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
# geojson_pd = pd.read_json(geojson_source)

# json_file = "etl/gz_2010_us_050_00_500k.json"
# customer_location_df = pd.read_json(json_file, encoding="utf-8")

In [39]:
results_file = "etl/countypres_2000-2016.csv"
results_df = pd.read_csv(results_file, encoding="utf-8")
results_df.head()
results_df.dropna(inplace=True)
results_df.drop(columns=['office', 'version'],inplace=True)
results_df.head()

,year,state,state_po,county,FIPS,candidate,party,candidatevotes,totalvotes
0,2000,Alabama,AL,Autauga,1001.0,Al Gore,democrat,4942.0,17208
1,2000,Alabama,AL,Autauga,1001.0,George W. Bush,republican,11993.0,17208
2,2000,Alabama,AL,Autauga,1001.0,Ralph Nader,green,160.0,17208
4,2000,Alabama,AL,Baldwin,1003.0,Al Gore,democrat,13997.0,56480
5,2000,Alabama,AL,Baldwin,1003.0,George W. Bush,republican,40872.0,56480


In [40]:
# results_df.loc[results_df['county'].str.lower() =="Orange".lower(),:]

# Cast float to integer
results_df.FIPS = results_df.FIPS.astype(int)
results_df.candidatevotes = results_df.candidatevotes.astype(int)
results_df.party = results_df.party.str.title()
# Calculate percent won
results_df["percentwon"] = ((results_df.candidatevotes / results_df.totalvotes) * 100).map("{0:,.2f}".format).astype(float)
# results_df.percentwon = results_df.percentwon.astype(int)
# Drop index
results_df.reset_index(drop=True, inplace=True)

# Test results
# results_df.loc[(results_df['county'].str.lower() =="Orange".lower()) & (results_df['state'] == "California"),:].head(100)
# results_df.head()
# results_2016 = results_df.loc[(results_df.year == 2016) & (results_df['state'] == "California"),:]
results_2016 = results_df.loc[results_df.year == 2016,:]

In [ ]:
results_2016

In [43]:
# results_df.to_json("etl/processed.json", orient='records')

## GEO JSON BIG SOURCE

In [38]:
# https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
# geojson_source = "etl/us-county-boundaries.geojson" https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?refine.stusab=CA
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
# geojson_source = "etl/ca-us-county-boundaries.csv.csv"
geojson_source = "etl/us-county-boundaries.csv"
geojson_pd = pd.read_csv(geojson_source, delimiter=';')
geojson_pd.dtypes
geojson_pd = geojson_pd[['GEOID','Geo Shape']]

### GEOJSON Small source

In [54]:
# https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
# geojson_source = "etl/us-county-boundaries.geojson" https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?refine.stusab=CA
# geojson_source = "etl/gz_2010_us_050_00_500k.json"
# geojson_source = "etl/ca-us-county-boundaries.csv.csv"
geojson_source = "etl/gz_2010_us_050_00_20m.json"
geojson_pd = pd.read_json(geojson_source, encoding='latin-1', lines=True)
geojson_pd.rename(columns={"geometry": "Geo Shape"},inplace=True)
# geojson_pd = geojson_pd[['Geometry']]
# geojson_pd['FIPS'] = int(geojson_pd['properties']['GEO_ID'][-5:])
geojson_pd['GEOID']  = geojson_pd.apply(lambda row: int(row.properties['GEO_ID'][-5:]), axis=1)
# geojson_pd['properties'][0]['GEO_ID'] '0500000US01001'
# int(geojson_pd['properties'][0]['GEO_ID'][-5:])
# geojson_pd

In [37]:
geojson_pd.head(10)

,properties,Geo Shape,GEOID
0,"{'GEO_ID': '0500000US01001', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.4967...",1001
1,"{'GEO_ID': '0500000US01009', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.5777...",1009
2,"{'GEO_ID': '0500000US01017', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-85.1841...",1017
3,"{'GEO_ID': '0500000US01021', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.5173...",1021
4,"{'GEO_ID': '0500000US01033', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-88.1399...",1033
5,"{'GEO_ID': '0500000US01045', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-85.4164...",1045
6,"{'GEO_ID': '0500000US01051', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.3367...",1051
7,"{'GEO_ID': '0500000US01065', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-87.4212...",1065
8,"{'GEO_ID': '0500000US01079', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-87.1050...",1079
9,"{'GEO_ID': '0500000US01083', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.8363...",1083


In [55]:
# geojson_pd
# results_df
joined_df = geojson_pd.merge(results_2016[['FIPS','year','state_po','county','candidate','party','candidatevotes','totalvotes','percentwon']].loc[(results_df.party == 'Republican')], how="inner", right_on='FIPS', left_on='GEOID')
joined_df = joined_df.merge(results_2016[['FIPS','candidate','party','candidatevotes','totalvotes','percentwon']].loc[(results_df.party == 'Democrat')], how="inner", right_on='FIPS', left_on='GEOID',suffixes=('_rep', '_dem'))
joined_df['winner'] = np.where(joined_df['candidatevotes_rep'] > joined_df['candidatevotes_dem'], 'red',  'blue')
joined_df

,properties,Geo Shape,GEOID,FIPS_rep,year,state_po,county,candidate_rep,party_rep,candidatevotes_rep,totalvotes_rep,percentwon_rep,FIPS_dem,candidate_dem,party_dem,candidatevotes_dem,totalvotes_dem,percentwon_dem,winner
0,"{'GEO_ID': '0500000US01001', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.4967...",1001,1001,2016,AL,Autauga,Donald Trump,Republican,18172,24973,72.77,1001,Hillary Clinton,Democrat,5936,24973,23.77,red
1,"{'GEO_ID': '0500000US01009', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.5777...",1009,1009,2016,AL,Blount,Donald Trump,Republican,22859,25588,89.33,1009,Hillary Clinton,Democrat,2156,25588,8.43,red
2,"{'GEO_ID': '0500000US01017', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-85.1841...",1017,1017,2016,AL,Chambers,Donald Trump,Republican,7843,13900,56.42,1017,Hillary Clinton,Democrat,5784,13900,41.61,red
3,"{'GEO_ID': '0500000US01021', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.5173...",1021,1021,2016,AL,Chilton,Donald Trump,Republican,15081,18369,82.10,1021,Hillary Clinton,Democrat,2911,18369,15.85,red
4,"{'GEO_ID': '0500000US01033', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-88.1399...",1033,1033,2016,AL,Colbert,Donald Trump,Republican,16746,24915,67.21,1033,Hillary Clinton,Democrat,7312,24915,29.35,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3110,"{'GEO_ID': '0500000US51001', 'STATE': '51', 'C...","{'type': 'MultiPolygon', 'coordinates': [[[[-7...",51001,51001,2016,VA,Accomack,Donald Trump,Republican,8583,15818,54.26,51001,Hillary Clinton,Democrat,6740,15818,42.61,red
3111,"{'GEO_ID': '0500000US51021', 'STATE': '51', 'C...","{'type': 'Polygon', 'coordinates': [[[-81.2251...",51021,51021,2016,VA,Bland,Donald Trump,Republican,2573,3139,81.97,51021,Hillary Clinton,Democrat,453,3139,14.43,red
3112,"{'GEO_ID': '0500000US51027', 'STATE': '51', 'C...","{'type': 'Polygon', 'coordinates': [[[-81.9682...",51027,51027,2016,VA,Buchanan,Donald Trump,Republican,7296,9247,78.90,51027,Hillary Clinton,Democrat,1721,9247,18.61,red
3113,"{'GEO_ID': '0500000US51037', 'STATE': '51', 'C...","{'type': 'Polygon', 'coordinates': [[[-78.4433...",51037,51037,2016,VA,Charlotte,Donald Trump,Republican,3479,5808,59.90,51037,Hillary Clinton,Democrat,2155,5808,37.10,red


In [48]:
# joined_df["margin_opacity"] = (abs(joined_df.percentwon_dem - joined_df.percentwon_rep)).map("{0:,.2f}".format).astype(float)

In [56]:
joined_df

,properties,Geo Shape,GEOID,FIPS_rep,year,state_po,county,candidate_rep,party_rep,candidatevotes_rep,totalvotes_rep,percentwon_rep,FIPS_dem,candidate_dem,party_dem,candidatevotes_dem,totalvotes_dem,percentwon_dem,winner
0,"{'GEO_ID': '0500000US01001', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.4967...",1001,1001,2016,AL,Autauga,Donald Trump,Republican,18172,24973,72.77,1001,Hillary Clinton,Democrat,5936,24973,23.77,red
1,"{'GEO_ID': '0500000US01009', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.5777...",1009,1009,2016,AL,Blount,Donald Trump,Republican,22859,25588,89.33,1009,Hillary Clinton,Democrat,2156,25588,8.43,red
2,"{'GEO_ID': '0500000US01017', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-85.1841...",1017,1017,2016,AL,Chambers,Donald Trump,Republican,7843,13900,56.42,1017,Hillary Clinton,Democrat,5784,13900,41.61,red
3,"{'GEO_ID': '0500000US01021', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.5173...",1021,1021,2016,AL,Chilton,Donald Trump,Republican,15081,18369,82.10,1021,Hillary Clinton,Democrat,2911,18369,15.85,red
4,"{'GEO_ID': '0500000US01033', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-88.1399...",1033,1033,2016,AL,Colbert,Donald Trump,Republican,16746,24915,67.21,1033,Hillary Clinton,Democrat,7312,24915,29.35,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3110,"{'GEO_ID': '0500000US51001', 'STATE': '51', 'C...","{'type': 'MultiPolygon', 'coordinates': [[[[-7...",51001,51001,2016,VA,Accomack,Donald Trump,Republican,8583,15818,54.26,51001,Hillary Clinton,Democrat,6740,15818,42.61,red
3111,"{'GEO_ID': '0500000US51021', 'STATE': '51', 'C...","{'type': 'Polygon', 'coordinates': [[[-81.2251...",51021,51021,2016,VA,Bland,Donald Trump,Republican,2573,3139,81.97,51021,Hillary Clinton,Democrat,453,3139,14.43,red
3112,"{'GEO_ID': '0500000US51027', 'STATE': '51', 'C...","{'type': 'Polygon', 'coordinates': [[[-81.9682...",51027,51027,2016,VA,Buchanan,Donald Trump,Republican,7296,9247,78.90,51027,Hillary Clinton,Democrat,1721,9247,18.61,red
3113,"{'GEO_ID': '0500000US51037', 'STATE': '51', 'C...","{'type': 'Polygon', 'coordinates': [[[-78.4433...",51037,51037,2016,VA,Charlotte,Donald Trump,Republican,3479,5808,59.90,51037,Hillary Clinton,Democrat,2155,5808,37.10,red


In [57]:
def df_to_geojson(df, properties):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{}
#                    'geometry':{}
                  }
        feature['geometry'] = row['Geo Shape'] #Use json.loads(row['Geo Shape']) if using the big source
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [58]:
# properties = ['year', 'state_po', 'county', 'candidate_rep', 'party_rep', 'candidatevotes_rep', 'percentwon_rep', 'candidate_dem', 'party_dem', 'candidatevotes_dem', 'totalvotes_dem', 'percentwon_dem', 'winner']
properties = ['county', 'state_po' ,'candidatevotes_rep', 'percentwon_rep', 'candidatevotes_dem', 'totalvotes_dem', 'percentwon_dem', 'winner']
geojson = df_to_geojson(joined_df, properties)

In [61]:
json = json.dumps(geojson)
f = open("etl/processed.json","w")
f.write(json)
f.close()

# output_filename = 'dataset.js'
# with open(output_filename, 'wb') as output_file:
#     output_file.write('var dataset = ')
#     json.dump(geojson, output_file, indent=2) 

In [27]:
geojson_pd

,properties,geometry,FIPS
0,"{'GEO_ID': '0500000US01001', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.4967...",1001
1,"{'GEO_ID': '0500000US01009', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.5777...",1009
2,"{'GEO_ID': '0500000US01017', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-85.1841...",1017
3,"{'GEO_ID': '0500000US01021', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-86.5173...",1021
4,"{'GEO_ID': '0500000US01033', 'STATE': '01', 'C...","{'type': 'Polygon', 'coordinates': [[[-88.1399...",1033
...,...,...,...
3216,"{'GEO_ID': '0500000US51001', 'STATE': '51', 'C...","{'type': 'MultiPolygon', 'coordinates': [[[[-7...",51001
3217,"{'GEO_ID': '0500000US51021', 'STATE': '51', 'C...","{'type': 'Polygon', 'coordinates': [[[-81.2251...",51021
3218,"{'GEO_ID': '0500000US51027', 'STATE': '51', 'C...","{'type': 'Polygon', 'coordinates': [[[-81.9682...",51027
3219,"{'GEO_ID': '0500000US51037', 'STATE': '51', 'C...","{'type': 'Polygon', 'coordinates': [[[-78.4433...",51037
